In [1]:
import pandas as pd

exchange = pd.read_csv("../data/nutrition_exchange.csv")

In [ ]:
# 栄養素変換に必要な部分のみ抽出
cleaned_exchange = exchange[["id","kcal","water","prot",\
    "chole","fat","fib","polyl","na","k","ca","mg","p","fe","zn","cu","mn",\
        "iodine","se","cr","mo","vitd","vitk","thla","ribf","nia","vitb12","fol",\
            "pantac","biot","vitc"]]
cleaned_exchange = cleaned_exchange.astype(float)
print(cleaned_exchange.dtypes)
cleaned_exchange.to_csv("../data/cleaned_nutrition_exchange.csv")

In [6]:
"""
栄養素ベクトルの生成
"""
import pandas as pd
import json
exchange_amount = pd.read_csv("../data/exchange_complete.csv",names=["id","name","additonal","unit","g","kind"])

with open("../data/cleaned_cookpad_ingredients.json","r") as f:
    fix_nameToid_recipe = json.load(f)

"""
名前で記載されている食材をid表記にする
"""
fixed_recipes = {} # 本当はリストで保存はしないが、都合がよいので辞書型にはしない

for recipe in fix_nameToid_recipe.items():
    fixed_recipe = {}
    for ingredient in recipe[1].items():
        tmp = exchange_amount[exchange_amount["name"] == ingredient[0]]
        tmp = tmp[~tmp.duplicated(subset="name")]
        fixed_recipe[str(tmp["id"].tolist()[0])] = ingredient[1]
    fixed_recipes[recipe[0]] = fixed_recipe

output = json.dumps(fixed_recipes,ensure_ascii=False,indent=2)

with open("../data/id_cookpad_recipes.json","w") as f:
    f.write(output)

import numpy as np
from tqdm import tqdm as tq

nutrition_list = []

for recipe in tq(fixed_recipes.items(), total=len(fixed_recipes)):
  nutrition = np.arange(30,dtype=float)
  for ingredient in recipe[1].items():
    tmp = cleaned_exchange[cleaned_exchange["id"] == float(ingredient[0])]
    nutrition += tmp[["kcal","water","prot",\
    "chole","fat","fib","polyl","na","k","ca","mg","p","fe","zn","cu","mn",\
        "iodine","se","cr","mo","vitd","vitk","thla","ribf","nia","vitb12","fol",\
            "pantac","biot","vitc"]].values[0] * ingredient[1] * 0.01
  nutrition_row = []
  nutrition_row.append(recipe[0])
  for digit in nutrition:
    nutrition_row.append(digit)
  nutrition_list.append(nutrition_row)

recipe_nutrition = pd.DataFrame(nutrition_list, columns=["id","kcal","water","prot",\
    "chole","fat","fib","polyl","na","k","ca","mg","p","fe","zn","cu","mn",\
        "iodine","se","cr","mo","vitd","vitk","thla","ribf","nia","vitb12","fol",\
            "pantac","biot","vitc"])
recipe_nutrition.to_csv('../data/recipe_nutrition_cookpad.csv')

100%|██████████| 24975/24975 [00:50<00:00, 495.40it/s]


In [5]:
"""
何人前かの情報で除算する処理
"""

recipes = pd.read_csv("../data/taken_recipes.csv")
recipes = recipes[["recipe_id","title","serving_for"]]
with open("../data/cookpad_recipe_ingredients.json","r") as f:
    recipe_amount = json.load(f)
  
import numpy as np
from tqdm import tqdm as tq
nutrition_list = []
cleaned_recipes ={}

for recipe in tq(recipe_amount.items(), total=len(recipe_amount)):
  
  serving_for = recipes[recipes["recipe_id"] == recipe[0]].values[0][2]
  tmp = {}
  for ingredient in recipe[1].items():
    tmp[ingredient[0]] = ingredient[1] / serving_for
  
  cleaned_recipes[recipe[0]] = tmp

output = json.dumps(cleaned_recipes,ensure_ascii=False,indent=2)
with open("../data/cleaned_cookpad_ingredients.json","w") as f:
    f.write(output)    

100%|██████████| 24975/24975 [11:14<00:00, 37.03it/s]


In [7]:
# 食材ベクトルの生成

import json
import numpy as np
import pandas as pd

ingredients_vec = {}
with open("../data/cleaned_cookpad_ingredients.json","r") as f:
    ingredients_data = json.load(f)

# 変換表の読み込み
exchange_data = pd.read_csv("../data/exchange_complete.csv")

for recipe in ingredients_data.items():
    tmp = np.zeros(499,dtype="float")
    for ingre in recipe[1].items():
        match_ingre = exchange_data.query('name == "{}"'.format(ingre[0]))
        tmp[match_ingre[0:1].kind_index] = ingre[1]
    ingredients_vec[recipe[0]] = tmp

np.save("./cookpad_content_vec.npy",ingredients_vec)

In [103]:
"""
rakutenデータの分量情報を抽出
"""
import re
from kanjize import kanji2int
import mojimoji
rakuten_recipes = pd.read_csv("../data/recipe_data/rakuten_scray_recipes.csv",names=["recipe_id","title","serving_for"])
rakuten_recipes["serving_for"] = rakuten_recipes["serving_for"].fillna("1")
rakuten_serving_for = []

for recipe in tq(rakuten_recipes.itertuples(),total=len(rakuten_recipes)):
    try:
        serving_for_num = re.match("[0-9０-９一二三四五六七八九十]*",recipe.serving_for)[0]
        serving_for_num = mojimoji.zen_to_han(serving_for_num)
        serving_for_num = kanji2int(serving_for_num)
        rakuten_serving_for.append(int(serving_for_num))
    except Exception as e:
        print(e,"\n",recipe.serving_for)
        break


100%|██████████| 1380094/1380094 [00:13<00:00, 102791.02it/s]


In [114]:
re.match("[0-9０-９一二三四五六七八九十]*","10人")[0]

'10'

In [118]:
rakuten_recipes["serving_for"] = rakuten_serving_for
rakuten_recipes.loc[rakuten_recipes["serving_for"] == 0,"serving_for"] = 1
rakuten_recipes.to_csv("../data/recipe_data/rakuten_recipes.csv")

In [119]:
rakuten_recipes.describe()

,serving_for
count,1.380094e+06
mean,2.717031e+00
std,1.342865e+01
min,1.000000e+00
25%,1.000000e+00
50%,2.000000e+00
75%,3.000000e+00
max,3.200000e+03


In [2]:
import json

with open("../data/recipe_data/cleaned_cookpad_recipes.json","r") as f:
    pick_id_recipes = json.load(f)

cookpad_id_list = []

for recipe in pick_id_recipes.items():
    cookpad_id_list.append(recipe[0])

with open("../data/recipe_data/cookpad_id_list.txt","w") as f:
    f.write(",".join(cookpad_id_list))